##Introduction to RecBole:

In this notebook, we will indtroduce RecBole, a comprehensive library for recommender systems that includes implementations of several well-knwon recommenders from various categories and can be quite useful for you in your experimentations and studies. 

RecBole is developed based on Python and PyTorch for reproducing and developing recommendation algorithms in a unified, comprehensive and efficient framework for research purpose. It can be installed from pip, conda and source, and easy to use.

Let's get acquainted with different capabilities of RecBole one by one. First things first, let's install RecBole using "pip":

In [ ]:
!pip install recbole

We also cover how you can use [Ray](https://docs.ray.io/en/latest/), an outstanding library used for several useful machine learning tasks, for performing hyperparameter tuning together with RecBole. Please note that even if you don't want to use Ray for hyperparameter tuning with RecBole, since some of the py files forming RecBole import this package, if you don't install it you might encounter ugly errors. To avoid this, we recommend installing Ray and trying to use this handy framework. 

In [ ]:
!pip install ray

Next, we need to import the necessary packages from RecBole and also import torch. Note that here, we are importing all general recommenders in recbole which might not be the most efficient approach, but since we want to explore several models, we almost need them all.

In [6]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model import general_recommender
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.utils import get_model, get_trainer
import torch

in order to train a recommender model using recbole, you should specify the name of the model that you want to train, and the dataset on which you want to train the model. Next, you pass this config to "create_dataset" function to obtain a preprocessed dataset which will then be split using the "data_preparation" function. Finally, using the "get_model" method, by passing the train split of the dataset and the configurations, you make a model object. By passing the obtained model and config to the "get_trainer" method, you can construct a trainer object, and calling the "fit" method of this object, training process takes place. The function outputs the best validation score based on your metric of interest specified in the config dictionary, as well as the validation result which is a more comprehensive dictionary including more evaluation metrics evaluated on the validation split.

In [8]:
# configurations initialization
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = Config(model='EASE', dataset='ml-100k')

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()

# write config info into log
logger.info(config)

# dataset creating and filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

# model loading and initialization

model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)


print(best_valid_score)
print(best_valid_result)


0.4201
OrderedDict([('recall@10', 0.2389), ('mrr@10', 0.4201), ('ndcg@10', 0.2563), ('hit@10', 0.7847), ('precision@10', 0.1755)])


By calling the "evaluate" method on the trainer object and passing in the test split of your dataset, you can obtain the test results.

In [ ]:
# model evaluation
test_result = trainer.evaluate(test_data)
print(test_result)

OrderedDict([('recall@10', 0.2805), ('mrr@10', 0.5277), ('ndcg@10', 0.3295), ('hit@10', 0.8197), ('precision@10', 0.2206)])


Now, let's specify a different model and experiment with it. As stated earlier, if you want to change the trained model, all you have to do is to specify a different value for the "model" key in your config dictionary. You can view a list of implemented models in the [RecBole](https://recbole.io/model_list.html) docs. Just make sure you are using the right model for your task of interest. There are four categories of models that recbole supports: general recommendation, sequential recommendation, context-aware recommendation and knowledge-based recommendation.

In [ ]:
config = Config(model='BPR', dataset='ml-100k')
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
logger.info(model)
# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)


print(best_valid_score)
print(best_valid_result)


0.3993
OrderedDict([('recall@10', 0.2207), ('mrr@10', 0.3993), ('ndcg@10', 0.2369), ('hit@10', 0.7582), ('precision@10', 0.1582)])


In [ ]:
test_result = trainer.evaluate(test_data)
print(test_result)

OrderedDict([('recall@10', 0.2387), ('mrr@10', 0.4477), ('ndcg@10', 0.2831), ('hit@10', 0.7561), ('precision@10', 0.1982)])


Let's have a closer look on config object. Config gives you the list of all knobs and levers that you have to control your recommender training process.

In [ ]:
config


General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = /usr/local/lib/python3.10/dist-packages/recbole/config/../dataset_example/ml-100k
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hy

You can easily try changing the default hyperparameters of the model or training process by accessing the corresponding key, value pair in the config dics. For instance, here, we will try changin the maximum number of epochs to 100, and the training batch size to 512 to see the influence on the training results.

In [ ]:
parameter_dict = {
    'epochs' : 100,
    'training_batch_size': 512
}

config = Config(model='BPR', dataset='ml-100k', config_dict=parameter_dict)
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
logger.info(model)
# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)


print(best_valid_score)
print(best_valid_result)

0.4026
OrderedDict([('recall@10', 0.2263), ('mrr@10', 0.4026), ('ndcg@10', 0.2411), ('hit@10', 0.7815), ('precision@10', 0.1639)])


In [ ]:
test_result = trainer.evaluate(test_data)
print(test_result)

OrderedDict([('recall@10', 0.2526), ('mrr@10', 0.4748), ('ndcg@10', 0.2919), ('hit@10', 0.7953), ('precision@10', 0.1998)])


Finally, let's experiment with another model, LightGCN.

In [ ]:
config = Config(model='LightGCN', dataset='ml-100k')
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
logger.info(model)
# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)


print(best_valid_score)
print(best_valid_result)


0.4007
OrderedDict([('recall@10', 0.2222), ('mrr@10', 0.4007), ('ndcg@10', 0.2398), ('hit@10', 0.7434), ('precision@10', 0.1613)])


In [ ]:
test_result = trainer.evaluate(test_data)
print(test_result)

OrderedDict([('recall@10', 0.2589), ('mrr@10', 0.4743), ('ndcg@10', 0.3004), ('hit@10', 0.7953), ('precision@10', 0.205)])


Also, instead of changing the config dict directly by changing its values, you can pass in the hyperparameters from a pre-saved yaml file. You can see the code for this usage below, but since we aren't using it here, it's commented out.



In [ ]:
#config = Config(model='BPR', dataset='ml-100k', config_file_list=['example.yaml'], config_dict=parameter_dict)

What if we want to develop a new model and want to still use RecBole for its dataset pipeline, evaluator, etc.? You can easily define your own customized model following the instructions [here](https://recbole.io/docs/developer_guide/customize_models.html). All you have to do is to inherit from the proper abstract class (if your new recommender can be taxonomized in one of the recommender taxonomies that Recbole supports) and implement the required functions in your own model class. 

# Using a New Dataset

In order to characterize most forms of the input data required by different recommendation tasks, RecBole designs an input data format called Atomic Files and you need to convert your raw data into Atomic Files format before data loading.
Recbole developers have created atomic files of 28 [commonly used datasets](https://recbole.io/dataset_list.html) that you can download and use them right away. 

In [ ]:
parameter_dict = {
    'epochs' : 10,
    'training_batch_size': 512
}

config = Config(model='EASE', dataset='ml-1m', config_dict=parameter_dict)
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
logger.info(model)
# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)


print(best_valid_score)
print(best_valid_result)

0.4194
OrderedDict([('recall@10', 0.1803), ('mrr@10', 0.4194), ('ndcg@10', 0.2393), ('hit@10', 0.7803), ('precision@10', 0.1834)])


However, this automatic downloading tool appears to work improperly for some of the datasets that they have provided. It seems to be more proper to download the dataset files from the [recbole website](https://recbole.io/dataset_list.html), make a folder with the name "dataset" in your working directory, and copy the files there. 

In [ ]:
test_result = trainer.evaluate(test_data)
print(test_result)

OrderedDict([('recall@10', 0.2049), ('mrr@10', 0.5096), ('ndcg@10', 0.3019), ('hit@10', 0.8131), ('precision@10', 0.2291)])


Conversion of your own datasets into the format that RecBole accepts is also possible. You have to make your data follow the format indicated [here](https://recbole.io/atomic_files.html).

##Evaluation

RecBole supports most of the commonly-used ranking-based and value-based evaluation metrics. The supported ranking-based metrics include HR (hit ratio), NDCG, MRR, recall, MAP and precision, and value-based metrics include AUC, logloss, MAE and RMSE. You can specify your metrics of interest in the "config" object shown above. 
Currently, RecBole doesn't support beyond ranking and value metrics such as diversity, serendipity, etc. If you are interested in evaluating a model with such metrics, you can save and load the trained model and evaluate it yourself. Alternatively, you can define your own metric by creating a new metric class that inherits from the AbstractMetric class of RecBole. You can find the guidance for performing this [here](https://recbole.io/docs/developer_guide/customize_metrics.html). For more convenience, you can see how GAUC is defined by using the discussed abstraction in the line 223 of [metrcis.py](https://github.com/RUCAIBox/RecBole/blob/master/recbole/evaluator/metrics.py) source code and define your own metric class accordingly.

As an example of using different evaluation metrics available in Recbole, let's look at recall, mrr, ndcg, etc. at 1, 3 and 5 instead of 10 for the previous model. All we need to do is to specify a new 'topk' in the config dictionary. You have to be careful to also indicate a new 'valid_metric' which is the metric that RecBole uses for validation because it has to be one of the metrics that you have indicated in 'topk' list. The default value for 'valid_metric' is 'MRR@10' so, if we omit 10 from the topk list, we will encounter an error.

In [ ]:
parameter_dict = {
    'epochs' : 10,
    'training_batch_size': 512,
    'topk' : [1, 3, 5],
    'valid_metric' : 'MRR@5'
}

config = Config(model='EASE', dataset='ml-1m', config_dict=parameter_dict)
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
logger.info(model)
# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)


print(best_valid_score)
print(best_valid_result)

0.4045
OrderedDict([('recall@1', 0.0312), ('recall@3', 0.0776), ('recall@5', 0.1133), ('mrr@1', 0.2623), ('mrr@3', 0.3761), ('mrr@5', 0.4045), ('ndcg@1', 0.2623), ('ndcg@3', 0.2444), ('ndcg@5', 0.238), ('hit@1', 0.2623), ('hit@3', 0.525), ('hit@5', 0.6498), ('precision@1', 0.2623), ('precision@3', 0.2341), ('precision@5', 0.2125)])


The same applies for evaluation on the test split:

In [ ]:
test_result = trainer.evaluate(test_data)
print(test_result)

OrderedDict([('recall@1', 0.0392), ('recall@3', 0.0908), ('recall@5', 0.1293), ('mrr@1', 0.3695), ('mrr@3', 0.4653), ('mrr@5', 0.4879), ('ndcg@1', 0.3695), ('ndcg@3', 0.3268), ('ndcg@5', 0.31), ('hit@1', 0.3695), ('hit@3', 0.5876), ('hit@5', 0.6864), ('precision@1', 0.3695), ('precision@3', 0.3094), ('precision@5', 0.2749)])


##Saving and Loading Models

When we run a trainer object from RecBole, it will save the best model parameters in training process and its corresponding config settings. If you want to save filtered dataset and split dataloaders, you can set parameter save_dataset and parameter save_dataloaders to True to save filtered dataset and split dataloaders. As an example, let us save the split datasets in the above example.

In [9]:
parameter_dict = {
    'epochs' : 10,
    'training_batch_size': 512,
    'topk' : [1, 3, 5],
    'valid_metric' : 'MRR@5', 
    'save_dataloader' : True,
    'save_dataset' : True
}

config = Config(model='EASE', dataset='ml-1m', config_dict=parameter_dict)
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)
model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
logger.info(model)
# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)


print(best_valid_score)
print(best_valid_result)

0.4064
OrderedDict([('recall@1', 0.0309), ('recall@3', 0.0775), ('recall@5', 0.1134), ('mrr@1', 0.2661), ('mrr@3', 0.3786), ('mrr@5', 0.4064), ('ndcg@1', 0.2661), ('ndcg@3', 0.2459), ('ndcg@5', 0.239), ('hit@1', 0.2661), ('hit@3', 0.5268), ('hit@5', 0.648), ('precision@1', 0.2661), ('precision@3', 0.2357), ('precision@5', 0.2136)])


Now in order to load the saved and filtered dataset and the best model according to our define validation metric, you can apply load_data_and_model() to get them.

In [10]:
from recbole.quick_start import load_data_and_model
loaded_config, loaded_model, loaded_dataset, loaded_train_data, loaded_valid_data, loaded_test_data = load_data_and_model(
    model_file='saved/EASE-Jun-10-2023_23-48-13.pth',)
# Here you can replace it by your model path.
# And you can also pass 'dataset_file' and 'dataloader_file' to this function.

In [16]:
print(loaded_test_data.config)


General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/ml-1m
checkpoint_dir = saved
show_progress = True
save_dataset = True
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [1, 3, 5]
valid_metric = MRR@5
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIEL

In [21]:
loaded_trainer

In [ ]:
loaded_trainer = get_trainer(loaded_config['MODEL_TYPE'], loaded_config['model'])(loaded_config, loaded_model)
test_result = loaded_trainer.evaluate(loaded_test_data)
print(test_result)

As you can see, this leads to an error! If you check the RecBole source code, on line 132 of trainer.py, you can find the reason! Recbole looks gets the local time when trying to load the model. You can adjust this by changing the source code by yourself. This an example of why you may want to fork the Recbole repository and change the code in confronting with such situations. Overall, Recbole is still a developing library and it's a good idea to verify the code you want to run instead of treating it as a black box.

## Hyperparamter Tuning with RecBole

Tuning hyperparameters is an essential part of training ML models and recommender systems are no exception. Recbole has an internal hyperparamter tuner, but we suggest you use more stable hyperparameter tuning packages like Ray or Wandb. In this notebook, we will show an example of using Ray for tuning your recommender's hyperparameters.

First, let's import an initialize a ray agentand import the necessary packages and files.

In [ ]:
from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function



In [ ]:
import ray
ray.init()

2023-06-10 14:47:01,564	INFO worker.py:1636 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.5.0


In [ ]:
import os
import numpy as np

from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import math

In order to perform hp tuning, we should have an objective function as the one defined below. Please note that we have adjusted this function to work with ray and the original objective function implemented in the recbole's repository doesn't report the validation metric to ray. If you want to tune on another metric, please make sure you are reporting it at the end of this function as well.

In [ ]:
def objective_function(config_dict=None, config_file_list=None):

    config = Config(config_dict=config_dict, config_file_list=config_file_list)
    #init_seed(config['seed'])
    #init_seed(np.random.RandomState(config['seed']))
    
    dataset = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)
    model_name = config['model']
    model = get_model(model_name)(config, train_data._dataset).to(config['device'])
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, verbose=False)
    test_result = trainer.evaluate(test_data)

    return {
        'model': model_name,
        'best_valid_score': best_valid_score,
        'valid_score_bigger': config['valid_metric_bigger'],
        'best_valid_result': best_valid_result,
        'test_result': test_result,
        'mrr@10': best_valid_result['mrr@10']
    }

Finally, define the ray_tune function and run the tuning jobs. We specify the model and dataset (and other training non-tuned hyperparameter configurations) in "example.yaml" file, and specify the tuned hyperparameters and their ranges and selection methods in the "hyper.test" file. Make sure that these files are in your current working directory.

In [ ]:
def ray_tune(params_file, config_file):

      output_file = 'hyper_example.result'
      params_file = (os.path.join(os.getcwd(), params_file))
      config_file_list = [os.path.join(os.getcwd(), config_file)]
      #ray.init()
      tune.register_trainable("train_func", objective_function)
      config = {}
      with open(params_file, "r") as fp:
        for line in fp:
            para_list = line.strip().split(" ")
            if len(para_list) < 3:
                continue
            para_name, para_type, para_value = (
                para_list[0],
                para_list[1],
                "".join(para_list[2:]),
            )
            if para_type == "choice":
                para_value = eval(para_value)
                config[para_name] = tune.choice(para_value)
            elif para_type == "uniform":
                low, high = para_value.strip().split(",")
                config[para_name] = tune.uniform(float(low), float(high))
            elif para_type == "quniform":
                low, high, q = para_value.strip().split(",")
                config[para_name] = tune.quniform(float(low), float(high), float(q))
            elif para_type == "loguniform":
                low, high = para_value.strip().split(",")
                config[para_name] = tune.loguniform(
                    math.exp(float(low)), math.exp(float(high))
                )
            else:
                raise ValueError("Illegal param type [{}]".format(para_type))
      scheduler = ASHAScheduler(
        metric="mrr@10", mode="max", max_t=10, grace_period=1, reduction_factor=2)
      local_dir = "./ray_log"
      result = tune.run(
        tune.with_parameters(objective_function, config_file_list=config_file_list),
        config=config,
        num_samples=5,
        log_to_file=output_file,
        scheduler=scheduler,
        local_dir=local_dir,
        resources_per_trial={"cpu": 1},
    )
    
      best_trial = result.get_best_trial("mrr@10", "max", "last")
      print("best params: ", best_trial.config)
      print("best result: ", best_trial.last_result)


In [ ]:
ray_tune('hyper.test', 'example.yaml')

/usr/local/lib/python3.10/dist-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the`RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(
2023-06-10 15:57:10,560	WARNING callback.py:144 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


== Status ==
Current time: 2023-06-10 15:57:10 (running for 00:00:00.19)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 0/2 CPUs, 0/0 GPUs
Result logdir: /content/ray_log/objective_function_2023-06-10_15-57-10
Number of trials: 5/5 (5 PENDING)
+--------------------------------+----------+-------+------------------+-----------------+
| Trial name                     | status   | loc   |   embedding_size |   learning_rate |
|--------------------------------+----------+-------+------------------+-----------------|
| objective_function_7565c_00000 | PENDING  |       |                8 |      0.428019   |
| objective_function_7565c_00001 | PENDING  |       |                8 |      0.207121   |
| objective_function_7565c_00002 | PENDING  |       |                8 |      0.00269814 |
| objective_function_7565c_00003 | PENDING  |       |               16 |      0.00566054 |
| objective_function_75

(pid=41646) 2023-06-10 15:57:24.339877: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


== Status ==
Current time: 2023-06-10 15:57:25 (running for 00:00:15.37)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 2.0/2 CPUs, 0/0 GPUs
Result logdir: /content/ray_log/objective_function_2023-06-10_15-57-10
Number of trials: 5/5 (5 PENDING)
+--------------------------------+----------+-------+------------------+-----------------+
| Trial name                     | status   | loc   |   embedding_size |   learning_rate |
|--------------------------------+----------+-------+------------------+-----------------|
| objective_function_7565c_00000 | PENDING  |       |                8 |      0.428019   |
| objective_function_7565c_00001 | PENDING  |       |                8 |      0.207121   |
| objective_function_7565c_00002 | PENDING  |       |                8 |      0.00269814 |
| objective_function_7565c_00003 | PENDING  |       |               16 |      0.00566054 |
| objective_function_

2023-06-10 15:59:29,719	INFO tune.py:1111 -- Total run time: 139.17 seconds (139.11 seconds for the tuning loop).


best params:  {'learning_rate': 0.005660537073988653, 'embedding_size': 16}
best result:  {'model': 'BPR', 'best_valid_score': 0.3862, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('recall@10', 0.2087), ('mrr@10', 0.3862), ('ndcg@10', 0.227), ('hit@10', 0.7466), ('precision@10', 0.1566)]), 'test_result': OrderedDict([('recall@10', 0.2273), ('mrr@10', 0.4594), ('ndcg@10', 0.2723), ('hit@10', 0.7646), ('precision@10', 0.1838)]), 'mrr@10': 0.3862, 'time_this_iter_s': 38.78921556472778, 'done': True, 'training_iteration': 1, 'trial_id': '7565c_00003', 'date': '2023-06-10_15-58-41', 'timestamp': 1686412721, 'time_total_s': 38.78921556472778, 'pid': 41680, 'hostname': '389c9d6dbd38', 'node_ip': '172.28.0.12', 'config': {'learning_rate': 0.005660537073988653, 'embedding_size': 16}, 'time_since_restore': 38.78921556472778, 'iterations_since_restore': 1, 'experiment_tag': '3_embedding_size=16,learning_rate=0.0057'}
